In [1]:
import os
import math
import datetime as dt
import pandas as pd
from itertools import accumulate
import numpy as np

In [2]:
file = "DATA_keep/SOXL_1min.txt"

In [3]:
df = pd.read_csv(file, names=['Date', 'Open', 'High', 'Low', 'Close', 'Volume'], header=None)

In [4]:
df = df[-80000:]

In [5]:
df['Date'] = df['Date'].map(lambda d: dt.datetime.strptime(d, '%Y-%m-%d %H:%M:%S'))

In [6]:
df['Day'] = df['Date'].map(lambda d: d.date())

In [7]:
df.drop_duplicates(inplace=True)

---
## Calculate day's and previous day's close for every row

In [8]:
df1 = df.copy()

# Remove the 8pm formal records
df1 = df1[df1['Date'].apply(lambda x: x.time() != dt.time(20, 0))]

df1 = pd.concat([df1, df1.shift(1)[['Day', 'Close']].rename(columns={'Day': 'Prev Day', 'Close': 'Prev Close'})], 
                axis=1)

df1 = df1.dropna()
df1['Prev Close'] = df1.apply(lambda row: row['Prev Close'] if row['Prev Day'] != row['Day'] else math.nan, axis=1)
df1[58:64];

In [9]:
df1['Prev Close'] = list(accumulate(df1['Prev Close'], lambda x, y: x if np.isnan(y) else y))
df1.dropna()
df1[550:560];

In [10]:
df1 = pd.concat([df1, df1.shift(-1)[['Day', 'Close']].rename(columns={'Day': 'Next Day', 'Close': 'Next Close'})], 
                axis=1)
df1['Day Close'] = df1.apply(lambda row: row['Close'] if row['Next Day'] != row['Day'] else math.nan, axis=1)
del df1['Next Close']
del df1['Next Day']
df1[58:64];

In [11]:
df1 = df1[::-1]

df1['Day Close'] = list(accumulate(df1['Day Close'], lambda x, y: x if np.isnan(y) else y))
df1 = df1[::-1]
df1 = df1.dropna()

df1[490:500]

,Date,Open,High,Low,Close,Volume,Day,Prev Day,Prev Close,Day Close
705350,2020-05-08 19:52:00,139.8500,139.8500,139.85,139.85,100,2020-05-08,2020-05-08,129.28,139.5
705351,2020-05-08 19:53:00,139.9900,140.0000,139.99,140.00,990,2020-05-08,2020-05-08,129.28,139.5
705352,2020-05-08 19:56:00,140.4999,140.4999,139.84,139.84,322,2020-05-08,2020-05-08,129.28,139.5
705353,2020-05-08 19:57:00,139.5000,139.5000,139.50,139.50,800,2020-05-08,2020-05-08,129.28,139.5
705355,2020-05-11 04:13:00,141.0000,141.0000,141.00,141.00,100,2020-05-11,2020-05-08,139.50,138.0
705356,2020-05-11 07:00:00,136.0000,136.0000,136.00,136.00,200,2020-05-11,2020-05-11,139.50,138.0
705357,2020-05-11 07:01:00,136.4900,136.4900,136.49,136.49,100,2020-05-11,2020-05-11,139.50,138.0
705358,2020-05-11 07:02:00,136.5000,136.5000,136.50,136.50,200,2020-05-11,2020-05-11,139.50,138.0
705359,2020-05-11 07:03:00,135.7800,137.8000,135.00,136.43,1250,2020-05-11,2020-05-11,139.50,138.0
705360,2020-05-11 07:07:00,136.4300,136.4300,136.43,136.43,1850,2020-05-11,2020-05-11,139.50,138.0


---
## Determine the trading hour, weekday, and nights since last trade

In [12]:
cuts = [dt.time(h, m) for (h,m) in [(0, 0), (9, 30), (10, 0), (11, 0), (12, 0), (13, 0), (14, 0), (15, 0), (16, 0)]]

def trading_hour(ts):
    time = ts.time()
    for index, cut in enumerate(cuts[::-1]):
        if time >= cut:
            return 8 - index

df1['Trading Hour'] = df1['Date'].apply(trading_hour)

df1['Weekday'] = df1['Day'].apply(dt.date.weekday)

df1['Paused Days'] = (df1['Day'] - df1['Prev Day']).apply(lambda x: x.days)

df1.head(2)

,Date,Open,High,Low,Close,Volume,Day,Prev Day,Prev Close,Day Close,Trading Hour,Weekday,Paused Days
704860,2020-05-08 04:00:00,133.0,133.0,133.0,133.0,615,2020-05-08,2020-05-07,129.28,139.5,0,4,1
704861,2020-05-08 04:34:00,131.8,131.8,131.8,131.8,100,2020-05-08,2020-05-08,129.28,139.5,0,4,0


---
## Compute Trading Hour Figures

In [13]:
df1['Price Range (m)'] = df1['High'] - df1['Low']

grouped = df1.groupby(['Day', 'Trading Hour']).agg({
    'Open': ['first'], 
    'Close': ['last'],
    'Low': ['min'],
    'High': ['max'],
    'Date': ['count'],
    'Day Close': ['first'],
    'Prev Close': ['first'],
    'Price Range (m)': ['mean'],
    'Weekday': ['first'],
    'Paused Days': ['first']
    })

grouped.columns = ['Open', 'Close', 'Low', 'High', 'Count', 'Day Close', 'Prev Close', 'Vola (m)', 'Weekday', 'Paused Days']

for col in ['Open', 'Close', 'Low', 'High']:
    grouped[col] = round(100.0 * (grouped[col] / grouped['Prev Close'] - 1), 2)
grouped['Vola (m)'] = round(1000 * grouped['Vola (m)'] / grouped['Prev Close'], 3)

grouped['Gain'] = round(grouped['Close'] - grouped['Open'], 2)

grouped = grouped.reset_index()
grouped[:19]

,Day,Trading Hour,Open,Close,Low,High,Count,Day Close,Prev Close,Vola (m),Weekday,Paused Days,Gain
0,2020-05-08,0,2.88,1.83,-0.22,2.88,59,139.50,129.28,0.973,4,1,-1.05
1,2020-05-08,1,1.90,3.91,0.87,4.28,30,139.50,129.28,5.398,4,0,2.01
2,2020-05-08,2,3.96,4.96,3.74,6.46,60,139.50,129.28,3.208,4,0,1.00
3,2020-05-08,3,4.81,5.94,4.25,6.68,60,139.50,129.28,1.873,4,0,1.13
4,2020-05-08,4,5.97,6.60,5.58,6.64,58,139.50,129.28,1.296,4,0,0.63
5,2020-05-08,5,6.64,5.04,4.33,6.76,59,139.50,129.28,1.420,4,0,-1.60
6,2020-05-08,6,4.92,5.92,4.22,6.13,60,139.50,129.28,1.395,4,0,1.00
7,2020-05-08,7,5.99,7.63,5.40,7.70,60,139.50,129.28,1.968,4,0,1.64
8,2020-05-08,8,7.50,7.91,6.83,8.68,48,139.50,129.28,0.838,4,0,0.41
9,2020-05-11,0,1.08,-3.76,-5.38,1.08,73,138.00,139.50,1.720,0,3,-4.84


---
## Flatten the trading hours

In [14]:
rows = []
for day in grouped['Day'].unique():

    df_day = grouped[grouped['Day'] == day]

    row = {
        col + f' {th}': df_day[df_day['Trading Hour'] == th][col].iloc[0]
        for th in df_day['Trading Hour']
        for col in ['Open', 'Close', 'Low', 'High', 'Gain', 'Vola (m)']
    }    
    row['Paused Days'] = df_day['Paused Days'].iloc[0]
    row['Weekday'] = df_day['Weekday'].iloc[0]
    row['Day Close'] = df_day['Day Close'].iloc[0]
    row['Trend 1d'] = df_day['Day Close'].iloc[0] / df_day['Prev Close'].iloc[0] - 1.0
    row['Day'] = df_day['Day'].iloc[0]
    rows.append(row)

res = pd.DataFrame.from_records(rows)
res

,Open 0,Close 0,Low 0,High 0,Gain 0,Vola (m) 0,Open 1,Close 1,Low 1,High 1,...,Close 8,Low 8,High 8,Gain 8,Vola (m) 8,Paused Days,Weekday,Day Close,Trend 1d,Day
0,2.88,1.83,-0.22,2.88,-1.05,0.973,1.90,3.91,0.87,4.28,...,7.91,6.83,8.68,0.41,0.838,1,4,139.50,0.079053,2020-05-08
1,1.08,-3.76,-5.38,1.08,-4.84,1.720,-3.33,-2.40,-3.88,-0.89,...,-1.08,-1.43,-0.36,-0.43,0.222,3,0,138.00,-0.010753,2020-05-11
2,1.74,2.54,1.74,2.90,0.80,0.523,2.54,0.13,-1.85,3.46,...,-10.69,-10.70,-7.30,-2.61,0.892,1,1,123.25,-0.106884,2020-05-12
3,1.66,5.14,-0.20,7.99,3.48,2.387,5.70,2.78,0.12,5.91,...,-0.24,-2.62,0.27,2.38,0.440,1,2,122.95,-0.002434,2020-05-13
4,-0.38,-4.64,-6.43,-0.33,-4.26,1.277,-4.64,-9.35,-9.72,-3.96,...,7.31,4.92,7.36,1.89,1.032,1,3,131.94,0.073119,2020-05-14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,0.09,-1.24,-1.74,0.09,-1.33,0.596,-1.17,-0.24,-2.63,0.08,...,-1.19,-1.24,-0.59,0.00,0.500,1,1,594.33,-0.011854,2021-02-09
191,1.29,2.60,1.29,2.60,1.31,0.377,2.64,3.57,0.97,3.78,...,0.53,0.53,0.95,-0.42,0.000,1,2,597.46,0.005266,2021-02-10
192,1.34,2.65,1.34,2.65,1.31,0.268,2.65,4.30,2.65,5.31,...,11.14,10.38,11.14,0.26,0.041,1,3,664.00,0.111371,2021-02-11
193,-0.75,-1.88,-1.95,-0.60,-1.13,0.580,-1.58,0.25,-2.85,0.37,...,3.40,2.44,3.61,0.00,0.231,1,4,686.60,0.034036,2021-02-12


---
## Compute trends (fractions - not percentage!)

In [15]:
for trend_len in [3, 10, 30, 90, 300]:
    res = pd.concat([res, res.shift(trend_len)['Day Close'].rename(f'Close-{trend_len}')], 
                    axis=1)
    res[f'Trend {trend_len}d'] = res['Day Close'] / res[f'Close-{trend_len}'] - 1.0
    del res[f'Close-{trend_len}']

#### Compute distances from moving averages
Use 10d, 20d, 50d, and 200d

In [16]:
smooth = 2.0

for length in [10, 50, 200]:
    res[f'ema{length}'] = list(accumulate(res['Day Close'], lambda x, y: x * (1-smooth/(1+length)) + y*smooth/(1+length) ))
    res[f'd_ema{length}'] = res['Day Close'] / res[f'ema{length}'] - 1
    del res[f'ema{length}']
res

,Open 0,Close 0,Low 0,High 0,Gain 0,Vola (m) 0,Open 1,Close 1,Low 1,High 1,...,Trend 1d,Day,Trend 3d,Trend 10d,Trend 30d,Trend 90d,Trend 300d,d_ema10,d_ema50,d_ema200
0,2.88,1.83,-0.22,2.88,-1.05,0.973,1.90,3.91,0.87,4.28,...,0.079053,2020-05-08,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000
1,1.08,-3.76,-5.38,1.08,-4.84,1.720,-3.33,-2.40,-3.88,-0.89,...,-0.010753,2020-05-11,NaN,NaN,NaN,NaN,NaN,-0.008815,-0.010335,-0.010647
2,1.74,2.54,1.74,2.90,0.80,0.523,2.54,0.13,-1.85,3.46,...,-0.106884,2020-05-12,NaN,NaN,NaN,NaN,NaN,-0.095893,-0.112072,-0.115368
3,1.66,5.14,-0.20,7.99,3.48,2.387,5.70,2.78,0.12,5.91,...,-0.002434,2020-05-13,-0.118638,NaN,NaN,NaN,NaN,-0.081716,-0.110247,-0.116489
4,-0.38,-4.64,-6.43,-0.33,-4.26,1.277,-4.64,-9.35,-9.72,-3.96,...,0.073119,2020-05-14,-0.043913,NaN,NaN,NaN,NaN,-0.011954,-0.043494,-0.051397
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,0.09,-1.24,-1.74,0.09,-1.33,0.596,-1.17,-0.24,-2.63,0.08,...,-0.011854,2021-02-09,0.059412,0.015949,0.336144,1.330706,NaN,0.041155,0.177925,0.740692
191,1.29,2.60,1.29,2.60,1.31,0.377,2.64,3.57,0.97,3.78,...,0.005266,2021-02-10,0.082355,0.216823,0.345631,1.212815,NaN,0.037837,0.175639,0.736899
192,1.34,2.65,1.34,2.65,1.31,0.268,2.65,4.30,2.65,5.31,...,0.111371,2021-02-11,0.103980,0.241121,0.512150,1.743802,NaN,0.122121,0.291050,0.912635
193,-0.75,-1.88,-1.95,-0.60,-1.13,0.580,-1.58,0.25,-2.85,0.37,...,0.034036,2021-02-12,0.155250,0.372267,0.487532,1.552416,NaN,0.127451,0.317682,0.958678


In [17]:
res.to_csv("somedata.csv", index=False)

In [18]:
cols = ["Gain 0", "Gain 1", "Gain 4", "Gain 8", "Vola (m) 0", "Paused Days", "Weekday", "Day Close", "Trend 1d", "Trend 30d", "Trend 300d", "d_ema10", "d_ema50", "d_ema200"]
res = pd.read_csv("somedata.csv")
res[cols]

,Gain 0,Gain 1,Gain 4,Gain 8,Vola (m) 0,Paused Days,Weekday,Day Close,Trend 1d,Trend 30d,Trend 300d,d_ema10,d_ema50,d_ema200
0,-1.05,2.01,0.63,0.41,0.973,1,4,139.50,0.079053,NaN,NaN,0.000000,0.000000,0.000000
1,-4.84,0.93,0.14,-0.43,1.720,3,0,138.00,-0.010753,NaN,NaN,-0.008815,-0.010335,-0.010647
2,0.80,-2.41,1.42,-2.61,0.523,1,1,123.25,-0.106884,NaN,NaN,-0.095893,-0.112072,-0.115368
3,3.48,-2.92,-1.29,2.38,2.387,1,2,122.95,-0.002434,NaN,NaN,-0.081716,-0.110247,-0.116489
4,-4.26,-4.71,3.35,1.89,1.277,1,3,131.94,0.073119,NaN,NaN,-0.011954,-0.043494,-0.051397
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,-1.33,0.93,-0.43,0.00,0.596,1,1,594.33,-0.011854,0.336144,NaN,0.041155,0.177925,0.740692
191,1.31,0.93,0.25,-0.42,0.377,1,2,597.46,0.005266,0.345631,NaN,0.037837,0.175639,0.736899
192,1.31,1.65,-0.40,0.26,0.268,1,3,664.00,0.111371,0.512150,NaN,0.122121,0.291050,0.912635
193,-1.13,1.83,0.27,0.00,0.580,1,4,686.60,0.034036,0.487532,NaN,0.127451,0.317682,0.958678


In [19]:
len(res.columns)

67

### Include relevant indexes' movements on the same scale
This may allow to identify rotations. Relevant indexes could include indexes representative of investment styles, like 'momentum', 'growth', 'value', 'dividend'


### Include macro-economic indicators
Like VIX, or even popular Buffet, Shiller or other indicators

Include USD-CHF, USD-Bitcoin 

Include other major markets. Hang Seng-based indices could be particularly predictive!!

Compute similarity to other pivotal market times, like Mar2020, 2008, 2000

### Include trend info from retail investors, like from wsb or robinhood
Also include FANG+